In [1]:
%pip install -U sentence-transformers datasets torch


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:

import torch
from sentence_transformers import SparseEncoder
from datasets import load_dataset
import numpy as np
from tqdm.auto import tqdm

# Загрузка модели SPLADE
print("Загрузка модели SPLADE...")
model = SparseEncoder("naver/splade-cocondenser-ensembledistil")

# Загрузка датасета MS MARCO (только маленькая часть для демонстрации)
print("Загрузка датасета MS MARCO...")
dataset = load_dataset("microsoft/ms_marco", "v1.1")

# Подготовка данных - берем первые 100 запросов для быстрого тестирования
queries = []
qids = []
documents = []
relevant_docs = {}

print("Подготовка данных...")
for i, example in enumerate(dataset['test']):
    query = example['query']
    passages = example['passages']
    
    if query and passages['passage_text']:
        queries.append(query)
        qids.append(example['query_id'])
        
        # Берем первые 10 пассажей для каждого запроса
        query_docs = passages['passage_text'][:10]
        documents.extend(query_docs)
        
        # Сохраняем релевантные документы для каждого запроса
        relevant_docs[example['query_id']] = [
            passages['passage_text'][j] for j in range(len(passages['is_selected'])) 
            if passages['is_selected'][j] == 1
        ]

print(f"Загружено {len(queries)} запросов и {len(documents)} документов")

# Функция для вычисления MRR@10
def calculate_mrr_at_10(queries, documents, relevant_docs, model, top_k=10):
    mrr_scores = []
    
    for i, query in enumerate(tqdm(queries, desc="Обработка запросов")):
        query_embedding = model.encode_query([query])
        
        # Вычисляем схожесть со всеми документами
        similarities = []
        batch_size = 32
        
        for j in range(0, len(documents), batch_size):
            batch_docs = documents[j:j+batch_size]
            doc_embeddings = model.encode_document(batch_docs)
            batch_similarities = model.similarity(query_embedding, doc_embeddings)
            similarities.extend(batch_similarities[0].cpu().numpy())
        
        # Получаем топ-K наиболее релевантных документов
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        top_documents = [documents[idx] for idx in top_indices]
        
        # Проверяем, есть ли релевантные документы в топ-10
        current_qid = qids[i]
        relevant_for_query = relevant_docs.get(current_qid, [])
        
        if relevant_for_query:
            reciprocal_rank = 0
            for rank, doc in enumerate(top_documents, 1):
                if doc in relevant_for_query:
                    reciprocal_rank = 1.0 / rank
                    break
            mrr_scores.append(reciprocal_rank)
        else:
            mrr_scores.append(0.0)
    
    return np.mean(mrr_scores)

# Вычисляем MRR@10
print("Вычисление MRR@10...")
mrr_score = calculate_mrr_at_10(queries, documents, relevant_docs, model, top_k=10)
print(f"MRR@10: {mrr_score:.4f}")

# Дополнительно: пример работы модели на одном запросе
print("\n--- Пример работы ---")
test_query = queries[0]
print(f"Запрос: {test_query}")

query_embedding = model.encode_query([test_query])
doc_embeddings = model.encode_document(documents[:5])

similarities = model.similarity(query_embedding, doc_embeddings)
print("Схожести с первыми 5 документами:")
for i, sim in enumerate(similarities[0]):
    print(f"Документ {i+1}: {sim:.4f}")

Загрузка модели SPLADE...
Загрузка датасета MS MARCO...
Подготовка данных...
Загружено 9650 запросов и 79175 документов
Вычисление MRR@10...


Обработка запросов:   0%|          | 0/9650 [00:00<?, ?it/s]


NotImplementedError: Could not run 'aten::as_strided' with arguments from the 'SparseMPS' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::as_strided' is only available for these backends: [CPU, MPS, Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradMAIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastMTIA, AutocastMAIA, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMo